In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as DriverWait
from selenium.webdriver.support import expected_conditions as DriverConditions
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import pandas as pd
from math import ceil

In [2]:
chrome_options = Options()
#отключает функции, которые могут использоваться для обнаружения автоматизации в Chrome
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
#отключает использование расширения автоматизации в браузере
chrome_options.add_experimental_option('useAutomationExtension', False)
#исключить определенные переключатели Chrome и предотвратить обнаружение автоматизированного трафика
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
#полный экран
chrome_options.add_argument("--start-maximized")

browser = Chrome("C:/Users/vital/OneDrive/Рабочий стол/chromedriver_win32/chromedriver", options=chrome_options)
#задаем неявное ожидание
browser.implicitly_wait(10)

In [3]:
url = 'https://www.classcentral.com/subjects'
browser.get(url)

In [4]:
soup = BeautifulSoup(browser.page_source, 'lxml')

In [5]:
#количество big_subjects
num = len(soup.find('ul', class_='filter-tabs__list').findAll('li'))

# собираем ссылки и названия big_subjects 
##так как ссылку на текущую страницу не показывают, то мы находимся на 1 subjects?topic=cs 
link_and_name_big_subjects = []
for i in range(num): 
    name_big_subjects = soup.find('ul', class_='filter-tabs__list').findAll('li')[i].text.strip()
    try:
        link_big_subjects = 'https://www.classcentral.com'+soup.find('ul', class_='filter-tabs__list').findAll('li')[i].find('a',class_='no-underline').get('href')
        link_and_name_big_subjects.append([link_big_subjects, name_big_subjects])
    except:
        link_big_subjects = "https://www.classcentral.com/subjects?topic=cs"
        link_and_name_big_subjects.append([link_big_subjects, name_big_subjects])
link_and_name_big_subjects

[['https://www.classcentral.com/subjects?topic=cs', 'Computer Science'],
 ['https://www.classcentral.com/subjects?topic=business', 'Business'],
 ['https://www.classcentral.com/subjects?topic=humanities', 'Humanities'],
 ['https://www.classcentral.com/subjects?topic=data-science', 'Data Science'],
 ['https://www.classcentral.com/subjects?topic=personal-development',
  'Personal Development'],
 ['https://www.classcentral.com/subjects?topic=art-and-design',
  'Art & Design'],
 ['https://www.classcentral.com/subjects?topic=programming-and-software-development',
  'Programming'],
 ['https://www.classcentral.com/subjects?topic=engineering', 'Engineering'],
 ['https://www.classcentral.com/subjects?topic=health', 'Health & Medicine'],
 ['https://www.classcentral.com/subjects?topic=maths', 'Mathematics'],
 ['https://www.classcentral.com/subjects?topic=science', 'Science'],
 ['https://www.classcentral.com/subjects?topic=social-sciences',
  'Social Sciences'],
 ['https://www.classcentral.com/subj

In [6]:

name_and_link_subjects = []
len_big_subjects = len(link_and_name_big_subjects)

for big_subjects in range(len_big_subjects):
    #переход по big_subjects
    browser.get(link_and_name_big_subjects[big_subjects][0])
    sleep(5)


    #по тексту последней кнопки искать сколько страниц 
    try:
        len_page = int(browser.find_elements_by_class_name('pagination__page')[-1].text)
    except:
        len_page = 0

    #на случай, если страница 1  
    #сбор ссылок и названий с страницы и указываем big_subjects
    soup = BeautifulSoup(browser.page_source, 'lxml')
    len_subjects = len(soup.find('ol', class_='l-subjects-page__subjects-list').findAll('li'))
    for i in range(len_subjects):
        link_subjects = 'https://www.classcentral.com' + soup.find('ol', class_='l-subjects-page__subjects-list').findAll('li')[i].find('a').get('href')
        name_subjects = soup.find('ol', class_='l-subjects-page__subjects-list').findAll('li')[i].find('a').find('span').text.strip()
        name_and_link_subjects.append([link_and_name_big_subjects[big_subjects][1],name_subjects,link_subjects])

    for num_page in range(1, len_page+1):
        #переход на нужную страницу
        #мы уже на 1 странице собрали
        if num_page == 1:
            continue
        else:
            browser.get(link_and_name_big_subjects[big_subjects][0] + f'&page={num_page}')
            sleep(5)
        #сбор ссылок и названий с страницы + первым название big_subjects, как на скрине 
        soup = BeautifulSoup(browser.page_source, 'lxml')
        len_subjects = len(soup.find('ol', class_='l-subjects-page__subjects-list').findAll('li'))
        for i in range(len_subjects):
            link_subjects = 'https://www.classcentral.com' + soup.find('ol', class_='l-subjects-page__subjects-list').findAll('li')[i].find('a').get('href')
            name_subjects = soup.find('ol', class_='l-subjects-page__subjects-list').findAll('li')[i].find('a').find('span').text.strip()
            name_and_link_subjects.append([link_and_name_big_subjects[big_subjects][1],name_subjects,link_subjects])
    

        
len(name_and_link_subjects)

1446

In [7]:
#создание промежуточного файла из-за большого количества курсов 260+ тысяч 
#в датафрейм
header = ["Big_subjects", "subjects", "subjects_url"]
df = pd.DataFrame(name_and_link_subjects, columns=header)
df

,Big_subjects,subjects,subjects_url
0,Computer Science,A* Search Algorithm,https://www.classcentral.com/subject/a-search-...
1,Computer Science,Active Directory,https://www.classcentral.com/subject/active-di...
2,Computer Science,Algorithms and Data Structures,https://www.classcentral.com/subject/algorithm...
3,Computer Science,Ansible,https://www.classcentral.com/subject/ansible
4,Computer Science,Apache Maven,https://www.classcentral.com/subject/apache-maven
...,...,...,...
1441,Certifications,SC-100: Microsoft Cybersecurity Architect,https://www.classcentral.com/subject/sc-100
1442,Certifications,SC-200: Microsoft Security Operations Analyst,https://www.classcentral.com/subject/sc-200
1443,Certifications,SC-300: Microsoft Identity and Access Administ...,https://www.classcentral.com/subject/sc-300
1444,Certifications,"SC-900: Microsoft Security, Compliance, and Id...",https://www.classcentral.com/subject/sc-900


In [8]:
df.to_excel('/Users/vital/OneDrive/Рабочий стол/classcentral.xlsx', index= False)

In [ ]:
##################################################################################################

In [ ]:
##########################  сбор информации по курсам по собранным ссылкам  ######################

In [ ]:
##################################################################################################

In [9]:
data = pd.read_excel('/Users/vital/OneDrive/Рабочий стол/classcentral.xlsx')

In [10]:
#если брать из промежуточного файла
name_and_link_subjects = [list(x) for x in data.to_numpy()]
name_and_link_subjects

[['Computer Science',
  'A* Search Algorithm',
  'https://www.classcentral.com/subject/a-search-algorithm'],
 ['Computer Science',
  'Active Directory',
  'https://www.classcentral.com/subject/active-directory'],
 ['Computer Science',
  'Algorithms and Data Structures',
  'https://www.classcentral.com/subject/algorithms-and-data-structures'],
 ['Computer Science',
  'Ansible',
  'https://www.classcentral.com/subject/ansible'],
 ['Computer Science',
  'Apache Maven',
  'https://www.classcentral.com/subject/apache-maven'],
 ['Computer Science',
  'Arch Linux',
  'https://www.classcentral.com/subject/arch'],
 ['Computer Science',
  'Arduino',
  'https://www.classcentral.com/subject/arduino'],
 ['Computer Science', 'Argo', 'https://www.classcentral.com/subject/argo'],
 ['Computer Science',
  'Artificial Intelligence',
  'https://www.classcentral.com/subject/ai'],
 ['Computer Science', 'Aruba', 'https://www.classcentral.com/subject/aruba'],
 ['Computer Science',
  'AutoML',
  'https://www.c

In [ ]:
#сбор данных по курсам

In [12]:
#проверка на рекламу
ad = 0
names = []
aaa = 110
len_cource = 0
for link in tqdm(range(len(name_and_link_subjects))):
    #перезапуск браузера на случай заполнения кэша
    if aaa > 100:
        aaa = 0
        browser.quit()
        browser = Chrome("C:/Users/vital/OneDrive/Рабочий стол/chromedriver_win32/chromedriver", options=chrome_options)
        browser.implicitly_wait(15)
    aaa += 1
    #переход по ссылкам subjects
    browser.get(name_and_link_subjects[link][2])
    sleep(2)
    #проверка на колличество курсов
    len_col = browser.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div/div/div[2]/span').text
    len_col = int(len_col[:len_col.find(' ')].replace(',', ''))
    len_cource = len_col + len_cource
    
    #на странице по 15 курсов, ищем количество страниц
    page = ceil(len_col/15)
    for page in range(page):
        if page != 0:
            browser.get(name_and_link_subjects[link][2]+f'?page={page+1}')
            sleep(5)
            aaa += 1
        #чтобы неявное ожидание сработало
        browser.find_element_by_xpath('/html/body/div[1]/div[3]/div[2]/div[1]/span').text
        
        soup = BeautifulSoup(browser.page_source, 'lxml')
        #border - все ячейки курсов, даже с рекламой
        border = soup.find('div', class_='catalog-grid__results').findAll('li', class_ = 'border-all')
        for i in range(len(border)):
            try:
                name_provider = border[i].find('a', class_='hover-underline').text.strip()
                Course_name = border[i].find('a', class_='course-name').text.strip()
                Course_url = 'https://www.classcentral.com' + border[i].find('a', class_='course-name').get('href')
                #название института может быть в виде картинки, текста или не быть вообще
                try:
                    #картинка
                    institution = border[i].find('span', class_='margin-right-xsmall').find('img').get('title')
                except:
                    try:
                        #текст
                        institution = border[i].find('span', class_='color-charcoal').text.strip()
                    except:
                        #вообще нет
                        institution = ''
                #сбор датасета
                names.append([name_and_link_subjects[link][0], name_and_link_subjects[link][1], name_and_link_subjects[link][2],Course_name, Course_url, name_provider, institution])
            except:
                #это была реклама, если нет border[i].find('a', class_='hover-underline').text.strip()
                ad += 1
    print(len(names), len_cource)
print(len(names), len_cource)


  0%|                                                                               | 1/1446 [00:08<3:19:35,  8.29s/it]

1 1


  0%|                                                                               | 2/1446 [00:26<5:35:24, 13.94s/it]

38 38


  0%|▏                                                                            | 3/1446 [05:15<55:59:45, 139.70s/it]

...

752 752


  0%|▎                                                                             | 6/1446 [06:17<19:01:48, 47.58s/it]

755 755


  0%|▎                                                                             | 6/1446 [06:33<26:14:03, 65.59s/it]


KeyboardInterrupt: 

In [13]:
#в датафрейм
header = ["Big_subjects", "subjects", "subjects_url","course_name","course_url","provider","institution"]
df = pd.DataFrame(names, columns=header)
df

,Big_subjects,subjects,subjects_url,course_name,course_url,provider,institution
0,Computer Science,A* Search Algorithm,https://www.classcentral.com/subject/a-search-...,Star Mob! (A* Search Algorithm) - Game Program...,https://www.classcentral.com/classroom/youtube...,YouTube,The Cherno
1,Computer Science,Active Directory,https://www.classcentral.com/subject/active-di...,SC-300: Implement Access Management for Apps,https://www.classcentral.com/course/microsoft-...,Microsoft Learn,Microsoft
2,Computer Science,Active Directory,https://www.classcentral.com/subject/active-di...,Plan and implement identity synchronization,https://www.classcentral.com/course/microsoft-...,Microsoft Learn,Microsoft
3,Computer Science,Active Directory,https://www.classcentral.com/subject/active-di...,Deploy and manage identity infrastructure,https://www.classcentral.com/course/microsoft-...,Microsoft Learn,Microsoft
4,Computer Science,Active Directory,https://www.classcentral.com/subject/active-di...,MS-500 part 1 - Implement and manage identity ...,https://www.classcentral.com/course/microsoft-...,Microsoft Learn,Microsoft
...,...,...,...,...,...,...,...
780,Computer Science,Arduino,https://www.classcentral.com/subject/arduino,"How to Build a ""Defuse the Bomb"" Escape Room P...",https://www.classcentral.com/classroom/youtube...,YouTube,Playful Technology
781,Computer Science,Arduino,https://www.classcentral.com/subject/arduino,Arduino Tutorial 2: Understanding How Light Em...,https://www.classcentral.com/classroom/youtube...,YouTube,Paul McWhorter
782,Computer Science,Arduino,https://www.classcentral.com/subject/arduino,Tutorial for Programming Software Interrupts o...,https://www.classcentral.com/classroom/youtube...,YouTube,Paul McWhorter
783,Computer Science,Arduino,https://www.classcentral.com/subject/arduino,Arduino GPS with Data Logger,https://www.classcentral.com/classroom/youtube...,YouTube,Paul McWhorter


In [14]:
df.to_excel('/Users/vital/OneDrive/Рабочий стол/classcentral_main.xlsx')